## Aufgabe 12.1.2 – MQTT-Client, Datenbank und Dashboard

In dieser Aufgabe wurde ein vollständiger MQTT-Datenfluss von der Speicherung der Daten bis zur Visualisierung umgesetzt.  
Daten werden über ein MQTT-Protokoll empfangen, in einer TinyDB Datenbank gespeichert, aufbereitet und schließlich über ein interaktives Dashboard dargestellt.

## Systemübersicht

Das System besteht aus folgenden Kernmodulen:

- `mqtt_client.py`: Verbindet sich mit dem MQTT-Broker und speichert empfangene Nachrichten in TinyDB. \
Zur Konfiguration des Clients wird die Datei `config.yaml` verwendet, die die Verbindungsdetails zum MQTT-Broker enthält.
- `database.py`: Stellt über ein Singleton-Muster Zugriff auf die Datenbank bereit (Referenz: MECH-B-3-SWD-SWD-ILV).
- `transform.py`: Extrahiert Flaschen-bezogene Daten (Füllstand, Vibration, Drop-Oszillation, Klassifikation) und bereitet sie als DataFrame auf.
- `dashboard.py`: Zeigt interaktiv den Schwingungsverlauf ausgewählter Flaschen an und hebt den Klassifikationsstatus ("is_cracked") hervor.

## Datenfluss & Formatierung

Die eingehenden Rohdaten enthalten unter anderem:

- `fill_level_grams` und `vibration-index` (jeweils für rot, grün, blau)
- `final_weight` (Waage)
- `drop_oscillation` (Zeitreihe bei Vereinzelung der Flasche)
- `is_cracked` (Label aus Ground Truth)

Im Modul `transform.py` werden alle Flaschen mit gültiger Flaschen-ID ("bottle") extrahiert, bereinigt und pro Flasche im Tidy-Format aggregiert. \
Die Mindestanzahl von 50 Oszillationswerten wird erst im Modul `dashboard.py` berücksichtigt, um in der Visualisierung nur vollständige Zeitreihen darzustellen.

## Beispielhafte Visualisierung

Die `drop_oscillation` wird über ein interaktives Dashboard mit Dash dargestellt. \
Es folgt ein Plot einer beliebigen Zeitreihe, für die die Daten gespeichert und aufbereitet wurden (Plot aus `dashboard.py`):

![Schwingungsverlauf](../images/Plot_Schwingungsverlauf.png)

## Fazit

Die entwickelte Datenverarbeitungskette – vom Empfang der MQTT-Nachrichten bis zur strukturierten Visualisierung – funktioniert zuverlässig und modular.  
Alle relevanten Flaschendaten werden gespeichert, bereinigt und im Tidy-Format für Analysen bereitgestellt.  
Das interaktive Dashboard ermöglicht eine gezielte Untersuchung des Schwingungsverlaufs einzelner Flaschen und liefert eine erste visuelle Einschätzung möglicher Defekte.

Die implementierte Lösung bildet eine robuste Grundlage für weiterführende Aufgaben wie z.B. die Klassifikation defekter Flaschen auf Basis der Drop-Oszillationsdaten.

## Erweiterung: Temperaturdaten-Zuordnung über Zeitstempel

Im Rahmen der Regressionsaufgabe wurde erkannt, dass zusätzlich zu Füllstand, Vibration und Drop-Oszillation auch die Temperaturdaten aus dem Topic `temperature` berücksichtigt werden sollten.  
Da diese Einträge keine Flaschen-ID enthalten, war eine zeitbasierte Zuordnung zu den Flaschen erforderlich.

Bei der Analyse der gespeicherten Daten (`database.json`) zeigte sich:
- Das Topic `dispenser_green` enthält 240 Flaschen mit Zeitstempel.
- Das Topic `temperature` enthält 720 Messwerte (3 pro Flasche).
- Die Zeitstempel von `temperature` und `dispenser_green` stimmen exakt überein (Verhältnis 3:1).

Daher wurde in `transform.py` eine Zeitstempel-basierte Zuordnung implementiert:  
Jeweils drei gleichzeitig mit dem `dispenser_green`-Eintrag aufgezeichnete Temperaturwerte (je einer pro Farbe: rot, grün, blau) wurden der entsprechenden Flasche zugewiesen.

Die resultierenden Features:
- `temperature_red`
- `temperature_green`
- `temperature_blue`

werden nun gemeinsam mit den übrigen Flaschendaten im aggregierten DataFrame ausgegeben und stehen für weitere Analysen oder Regressionsmodelle zur Verfügung.